In [271]:
import os
import polars as pl
import altair as alt
import warnings

pl.Config(tbl_rows=100)

df = pl.read_parquet(os.path.join("data","fdb.parquet")).with_columns(
    (pl.col("datum").dt.year().cast(str) + pl.lit("-") + pl.col("datum").dt.month().cast(str) + pl.lit("-") + pl.col("datum").dt.day().cast(str)).alias("den")
)
vanocni = pl.read_json(os.path.join("data","vanocni_filmy_kompletni_metadata_vyber.json"))

from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings("ignore")

In [399]:
vysledky = pl.read_csv("data/rozstrel-cigler/result1.csv").rename({"":"nazev"}).with_columns(pl.col("nazev").str.slice(0,5).alias("zkratka")).join(
    vanocni.with_columns(pl.col("nazev").str.slice(0,5).alias("zkratka")),
    on='zkratka',
    how='left'
).join(
    pl.read_csv("data_raw/vanocni_filmy_datum.csv").with_columns(pl.col("nazev").str.slice(0,5).alias("zkratka")).drop("nazev"),
    on="zkratka",
    how="left"
).with_columns(
        pl.col("nazev").str.replace("Dalskabáty, hříšná ves aneb Zapomenutý čert","Dalskabáty, hříšná ves").replace("Sám doma 2: Ztracen v New Yorku","Sám doma 2").replace("Veselé Vánoce přejí chobotnice","Veselé Ván. přejí chobotnice").replace('O princezně Jasněnce a létajícím ševci', 'O princ. Jasněnce a lét. ševci').replace("O kominickém učni a dceři cukráře","O komin. učni a dceři cukráře")
).with_columns(
    pl.when(pl.col("zeme").str.contains("Česk")).then(pl.lit("český / československý")).otherwise(pl.lit("zahraniční")).alias("puvod")
).unique(subset="nazev",keep="first").sort(
    by="Estimate",descending=True
).with_row_index(offset=1)

In [401]:
vysledky

index,nazev,Estimate,Std. Error,z value,Pr(>|z|),low,up,csfd,pocet_hodnoceni,zkratka,url,format,id_puvodni,nazev_right,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni_right,id,weighted_avg_kolikateho,puvod
u32,str,f64,f64,str,str,f64,f64,i64,i64,str,str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64,i64,f64,str
1,"""Pelíšky""",0.231184,0.024447,"""9.4565324468077""","""3.18324484754975e-21""",0.183268,0.2791,91,117084,"""Pelíš""","""https://www.csfd.cz/film/4570-pelisky/prehled/""","""film""","""4570-pelisky""","""Pelíšky""","[""komedie"", ""drama""]","""Česko""",1999,116,"[""Jan Hřebejk""]","[""Petr Jarchovský""]","[""Miroslav Donutil"", ""Jiří Kodet"", … ""více""]","""Příběhy jedné historické generace - stárnoucích rodičů, dospívající mládeže a malých dětí. Děj je zasazen do konce šedesátých let - podzim 67 až léto …",91,117084,14,24.0,"""český / československý"""
2,"""S čerty nejsou žerty""",0.182383,0.024259,"""7.51802629258278""","""5.56093211572443e-14""",0.134835,0.229932,89,80866,"""S čer""","""https://www.csfd.cz/film/821-s-certy-nejsou-zerty/prehled/""","""film""","""821-s-certy-nejsou-zerty""","""S čerty nejsou žerty""","[""pohádka"", ""komedie""]","""Československo""",1984,91,"[""Hynek Bočan""]","[""Jiří Just"", ""Hynek Bočan""]","[""Luděk Munzar"", ""Vladimír Dlouhý"", … ""více""]","""Chamtivá macecha připraví svého nevlastního syna Petra o všechno, včetně dědictví po jeho otci. Petr se mezitím seznámí s dcerami místního knížete, kd…",89,80866,21,26.821429,"""český / československý"""
3,"""Tři oříšky pro Popelku""",0.0,0.01,"""NA""","""NA""",-0.0196,0.0196,86,73324,"""Tři o""","""https://www.csfd.cz/film/9430-tri-orisky-pro-popelku/prehled/""","""film""","""9430-tri-orisky-pro-popelku""","""Tři oříšky pro Popelku""","[""pohádka""]","""Československo / Východní Německo""",1973,75,"[""Václav Vorlíček""]","[""František Pavlíček"", ""Václav Vorlíček""]","[""Libuše Šafránková"", ""Pavel Trávníček"", … ""více""]","""Popelka je milé hodné děvče bez matky a posléze i bez otce, který ji nechá napospas své druhé ženě a její dceři. Obě nemají Popelku rády a mimo jiné s…",86,73324,26,25.208333,"""český / československý"""
4,"""Anděl Páně""",-0.327379,0.02319,"""-14.1171006525761""","""2.9800176704971e-45""",-0.372832,-0.281926,77,40797,"""Anděl""","""https://www.csfd.cz/film/215414-andel-pane/prehled/""","""film""","""215414-andel-pane""","""Anděl Páně""","[""pohádka""]","""Česko""",2005,91,"[""Jiří Strach""]","[""Lucie Konášová""]","[""Ivan Trojan"", ""Jiří Dvořák"", … ""více""]","""Je časné ráno, v předvečer Štědrého dne, a celé nebe se chystá na velkolepé oslavy Ježíškových narozenin. Panna Marie (Klára Issová) se svatou Veronik…",77,40797,1,23.066667,"""český / československý"""
5,"""S tebou mě baví svět""",-0.35515,0.023248,"""-15.2766731296738""","""1.09384119899802e-52""",-0.400715,-0.309584,82,44669,"""S teb""","""https://www.csfd.cz/film/7366-s-tebou-me-bavi-svet/prehled/""","""film""","""7366-s-tebou-me-bavi-svet""","""S tebou mě baví svět""","[""komedie"", ""rodinný""]","""Československo""",1982,82,"[""Marie Poledňáková""]","[""Marie Poledňáková""]","[""Július Satinský"", ""Jana Šulcová"", … ""více""]","""Každoroční pánská jízda tří otců se vlivem razantního zásahu manželek stává něčím jako mateřskou školou v zimní přírodě. Ukáže se, že v jistých situac…",82,44669,20,27.363636,"""český / československý"""
6,"""Na samotě u lesa""",-0.362666,0.023252,"""-15.5971101587519""","""7.61645629933175e-55""",-0.40824,-0.317092,88,66837,"""Na sa""","""https://www.csfd.cz/film/6663-na-samote-u-lesa/prehled/""","""film""","""6663-na-samote-u-lesa""","""Na samotě u lesa""","[""komedie"", ""rodinný"", ""drama""]","""Československo""",1976,93,"[""Jiří Menzel""]","[""Zdeněk Svěrák"", ""Ladislav Smoljak""]","[""Josef Kemr"", ""Zdeněk Svěrák"", … ""více""]","""Hrdiny jsou manželé Oldřich a Věra Lavičkovi, kteří se svými dětmi Petříkem a Zuzankou odjíždějí do Loukova na chalupu sv

In [443]:
alt.Chart(
    vysledky,
    title=alt.Title(
        "Výsledky rozstřelu vánočních filmů na iROZHLAS.cz",
        subtitle=["30 snímků vysílaných v letech 2007 až 2024 na nejsledovanějších","stanicích disproporčně často o vánočních prázdninách."]
    )
).mark_bar().encode(
    alt.Y("nazev:N", axis=alt.Axis(grid=True, title=None), sort=vysledky.sort(by="Estimate",descending=True).select(pl.col("nazev")).to_series().to_list()),
    alt.X("low:Q", axis=alt.Axis(grid=False, title=None)),
    alt.X2("up:Q"),
    alt.Color("puvod:N",scale=alt.Scale(range=['#4D688C','#B25342']), legend=alt.Legend(orient='top-left',title=None,fillColor='white'))
).configure_view(
    stroke='transparent'
)

alt.Chart(...)

In [145]:
vanocni.select(pl.col('nazev').unique()).sort(by='nazev')

nazev
str
"""Adéla ještě nevečeřela"""
"""Anděl Páně"""
"""Bohouš"""
"""Byl jednou jeden král..."""
"""Císařův pekař"""
"""Dalskabáty, hříšná ves aneb Zapomenutý čert"""
"""Dařbuján a Pandrhola"""
"""Dobrý voják Švejk"""
"""Grinch"""


In [421]:
vysledky.columns

['index',
 'nazev',
 'Estimate',
 'Std. Error',
 'z value',
 'Pr(>|z|)',
 'low',
 'up',
 'csfd',
 'pocet_hodnoceni',
 'zkratka',
 'url',
 'format',
 'id_puvodni',
 'nazev_right',
 'zanry',
 'zeme',
 'rok',
 'delka',
 'rezie',
 'scenar',
 'hraji',
 'anotace',
 'hodnoceni',
 'pocet_hodnoceni_right',
 'id',
 'weighted_avg_kolikateho',
 'puvod']

In [435]:
alt.Chart(
    vysledky,
    width=800,
    height=800
).mark_point(
).encode(
    alt.X("Estimate:Q"),
    alt.Y("hodnoceni:Q"),
    alt.Size("pocet_hodnoceni:Q")
) + alt.Chart(
    vysledky
).mark_text(
).encode(
    alt.X("Estimate:Q"),
    alt.Y("hodnoceni:Q"),
    alt.Text("nazev:N")
)

alt.LayerChart(...)

In [148]:
alt.Chart(
    vysledky
).mark_point(
).encode(
    alt.Y("Estimate:Q"),
    alt.X("weighted_avg_kolikateho:Q",scale=alt.Scale(domain=[23,31]))
)

alt.Chart(...)

In [351]:
nahrazeni = {
    "dekada_":"dekáda: ",
    "hraji_":"hraje: ",
    "zeme_":"země: ",
    "zanry_":"žánr: ",
    "_":" "
}

vysledky_atributy = pl.read_csv("data/rozstrel-cigler/result2.csv").rename({"":"atribut"}).with_columns(
    pl.col("atribut").str.replace_many(
        list(nahrazeni.keys()), list(nahrazeni.values())
    )
).filter(~pl.col("atribut").is_in(["delka","format film"])).with_columns(
    pl.when(
        pl.col('atribut').str.contains('hraje')
    ).then(
        pl.lit('obsazení')
    ).when(
        pl.col("atribut").str.contains("dekáda")
    ).then(
        pl.lit("dekáda")
    ).when(
        pl.col("atribut").str.contains("žánr")
    ).then(
        pl.lit("žánr")
    ).when(
        pl.col("atribut").str.contains("země")
    ).then(
        pl.lit("původ")
    ).alias("kategorie")
)

In [353]:
vysledky_atributy

atribut,Estimate,Std. Error,z value,Pr(>|z|),low,up,kategorie
str,f64,f64,f64,f64,f64,f64,str
"""hraje: Václav Švec""",-1.395877,0.123189,-11.331175,9.1963e-30,-1.637327,-1.154426,"""obsazení"""
"""země: USA""",-1.176604,0.207984,-5.657197,1.5387e-8,-1.584252,-0.768956,"""původ"""
"""hraje: Miloš Kopecký""",-0.294282,0.082532,-3.565673,0.000363,-0.456044,-0.132519,"""obsazení"""
"""žánr: pohádka""",-0.285753,0.05133,-5.566946,2.5924e-8,-0.386361,-0.185146,"""žánr"""
"""hraje: Naďa Konvalinková""",-0.192491,0.055251,-3.48393,0.000494,-0.300783,-0.084199,"""obsazení"""
"""země: Česko""",-0.154575,0.192449,-0.8032,0.421859,-0.531776,0.222625,"""původ"""
"""dekáda: 1970""",-0.112919,0.068561,-1.646986,0.099561,-0.247299,0.021461,"""dekáda"""
"""hraje: Oldřich Vlach""",-0.02439,0.066865,-0.364768,0.715285,-0.155445,0.106665,"""obsazení"""
"""žánr: rodinný""",-0.022655,0.050333,-0.450108,0.652632,-0.121308,0.075998,"""žánr"""


In [441]:
alt.Chart(
    vysledky_atributy,
    title=alt.Title(
        "Žádoucí a nežádoucí vlastnosti vánočních filmů",
        subtitle=[
            "Srovnání vychází z klikacího rozstřelu 30 vybraných filmů.",
            "Zahrnuje pouze vlastnosti společné čtyřem nebo více snímkům."
        ]
    )
).mark_bar(color= '#778C63').encode(
    alt.Y("atribut:N", axis=alt.Axis(grid=True, title=None),sort=vysledky_atributy.sort(by="Estimate",descending=True).select(pl.col("atribut")).to_series().to_list()),
    alt.X("low:Q", axis=alt.Axis(grid=False, title=None)),
    alt.X2("up:Q"),
    alt.Color("kategorie:N", legend=alt.Legend(orient='top-left',title=None,fillColor='white'), scale=alt.Scale(range=['#B25342', '#4D688C', '#778C63', '#B6995D']))
).configure_view(
    stroke='transparent'
)

alt.Chart(...)

In [413]:
top3 = vysledky.sort(by="Estimate",descending=True).head(3).select(pl.col("nazev")).to_series().to_list()
top4 = vysledky.sort(by="Estimate",descending=True).head(4).select(pl.col("nazev")).to_series().to_list()
top5 = vysledky.sort(by="Estimate",descending=True).head(5).select(pl.col("nazev")).to_series().to_list()
top6 = vysledky.sort(by="Estimate",descending=True).head(6).select(pl.col("nazev")).to_series().to_list()

In [415]:
df.filter(pl.col("nazev").is_in(top6)).sort(by="datum").group_by(pl.col("den")).len().filter(pl.col("len") >= 3)

den,len
str,u32
"""2018-12-24""",3
"""2014-12-24""",3
"""2019-12-28""",3
"""2016-12-24""",3
"""2018-12-26""",3
"""2019-12-26""",3
"""2016-12-23""",3
"""2013-12-24""",3
"""2022-12-23""",3


In [447]:
df.filter(pl.col("nazev").is_in(vysledky.select(pl.col("nazev")).to_series().to_list())).sort(by="datum").group_by(pl.col("den")).len().filter(pl.col("len") >= 3).sort(by="len",descending=True)

den,len
str,u32
"""2018-12-24""",14
"""2014-12-24""",13
"""2020-12-24""",12
"""2021-12-24""",12
"""2023-12-24""",12
"""2020-12-26""",12
"""2021-12-27""",12
"""2016-12-24""",12
"""2013-12-24""",11


In [273]:
df.filter(pl.col("nazev").is_in(top3)).sort(by="datum").group_by(pl.col("den")).len().filter(pl.col("len") >= 3)

den,len
str,u32
"""2014-12-24""",3
"""2015-12-25""",3
"""2016-12-26""",3
"""2016-12-24""",3
"""2013-12-24""",3


In [277]:
df.filter(pl.col("den") == "2014-12-24").filter(pl.col("nazev").is_in(top3))

kanal,datum,nazev,extra,noticka,url,den
cat,datetime[μs],str,str,str,str,str
"""Prima""",2014-12-24 20:15:00,"""Tři oříšky pro Popelku""","""Pohádka ČR (1973). O chudé Popelce, která díky kouzelným oříškům a své chytrosti ke štěstí přišla.""",null,"""film/tri-orisky-pro-popelku/21407""","""2014-12-24"""
"""ČT1""",2014-12-24 20:20:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesá…","""film/pelisky/25517""","""2014-12-24"""
"""VOYO""",2014-12-24 17:15:00,"""Tři oříšky pro Popelku""",null,"""Film ČR/N (1973). Krásnou, ale chudou Libušku Šafránkovou nečekají v kouzelných oříšcích jen plesové šaty, ale také lovecká kamizola... A princ Pavel …","""film/tri-orisky-pro-popelku/21407""","""2014-12-24"""


In [279]:
df.filter(pl.col("den") == "2015-12-25").filter(pl.col("nazev").is_in(top3))

kanal,datum,nazev,extra,noticka,url,den
cat,datetime[μs],str,str,str,str,str
"""VOYO""",2015-12-25 20:00:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR/N (1973). Krásnou, ale chudou Popelku nečekají v kouzelných oříšcích jen plesové šaty, ale také lovecká kamizola... A princ na ní může oči …","""film/tri-orisky-pro-popelku/21407""","""2015-12-25"""
"""VOYO""",2015-12-25 16:30:00,"""S čerty nejsou žerty""","""Pohádka ČR (1984). Není jednoduché dostat do pekla pravou hříšnici!""",null,"""film/s-certy-nejsou-zerty/18331""","""2015-12-25"""
"""NOVA""",2015-12-25 20:20:00,"""S čerty nejsou žerty""","""19.04.2025 v 20:15 (Prima)21.04.2025 v 14:45 (Prima)23.04.2025 v 20:00 (Prima MAX)""","""Další díl/opakování:19.04.2025 v 20:15 (Prima)21.04.2025 v 14:45 (Prima)23.04.2025 v 20:00 (Prima MAX)""","""film/s-certy-nejsou-zerty/18331""","""2015-12-25"""


In [281]:
df.filter(pl.col("den") == "2016-12-26").filter(pl.col("nazev").is_in(top3))

kanal,datum,nazev,extra,noticka,url,den
cat,datetime[μs],str,str,str,str,str
"""NOVA""",2016-12-26 11:05:00,"""S čerty nejsou žerty""","""19.04.2025 v 20:15 (Prima)21.04.2025 v 14:45 (Prima)23.04.2025 v 20:00 (Prima MAX)""","""Další díl/opakování:19.04.2025 v 20:15 (Prima)21.04.2025 v 14:45 (Prima)23.04.2025 v 20:00 (Prima MAX)""","""film/s-certy-nejsou-zerty/18331""","""2016-12-26"""
"""VOYO""",2016-12-26 12:15:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR/N (1973). Krásnou, ale chudou Popelku nečekají v kouzelných oříšcích jen plesové šaty, ale také lovecká kamizola... A princ na ní může oči …","""film/tri-orisky-pro-popelku/21407""","""2016-12-26"""
"""ČT1""",2016-12-26 15:55:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR (1973). Klasická pohádka na motivy Boženy Němcové o krásné osiřelé dívce a kouzelném dárku. Scénář F. Pavlíček a V. Vorlíček. Kamera J. Ill…","""film/tri-orisky-pro-popelku/21407""","""2016-12-26"""


In [283]:
df.filter(pl.col("den") == "2016-12-24").filter(pl.col("nazev").is_in(top3))

kanal,datum,nazev,extra,noticka,url,den
cat,datetime[μs],str,str,str,str,str
"""ČT1""",2016-12-24 21:45:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci še…","""film/pelisky/25517""","""2016-12-24"""
"""VOYO""",2016-12-24 16:55:00,"""S čerty nejsou žerty""","""Pohádka ČR (1984). Není jednoduché dostat do pekla pravou hříšnici!""",null,"""film/s-certy-nejsou-zerty/18331""","""2016-12-24"""
"""ČT1""",2016-12-24 20:20:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR (1973). Opakování 26.12./ČT1. Klasická pohádka na motivy Boženy Němcové o krásné osiřelé dívce a kouzelném dárku. Scénář F. Pavlíček a V. V…","""film/tri-orisky-pro-popelku/21407""","""2016-12-24"""


In [285]:
df.filter(pl.col("den") == "2013-12-24").filter(pl.col("nazev").is_in(top3))

kanal,datum,nazev,extra,noticka,url,den
cat,datetime[μs],str,str,str,str,str
"""ČT1""",2013-12-24 20:30:00,"""Pelíšky""",null,"""Film ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesátých let…","""film/pelisky/25517""","""2013-12-24"""
"""VOYO""",2013-12-24 12:45:00,"""Tři oříšky pro Popelku""",null,"""Film ČR/N (1973). Krásnou, ale chudou Libušku Šafránkovou nečekají v kouzelných oříšcích jen plesové šaty, ale také lovecká kamizola... A princ Pavel …","""film/tri-orisky-pro-popelku/21407""","""2013-12-24"""
"""NOVA""",2013-12-24 20:20:00,"""Tři oříšky pro Popelku""",null,"""Krásnou, ale chudou Popelku nečekají v kouzelných oříšcích jen plesové šaty, ale také lovecká kamizola... A princ na ní může oči nechat!""","""film/tri-orisky-pro-popelku/21407""","""2013-12-24"""


In [257]:
df.filter(pl.col("kanal") == "ČT1").filter(pl.col("nazev").is_in(top3)).sort(by="datum")

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2011-03-08 20:55:00,"""Pelíšky""",null,null,"""film/pelisky/25517"""
"""ČT1""",2011-12-24 21:25:00,"""Pelíšky""",null,null,"""film/pelisky/25517"""
"""ČT1""",2012-12-24 20:35:00,"""Pelíšky""",null,null,"""film/pelisky/25517"""
"""ČT1""",2013-12-24 20:30:00,"""Pelíšky""",null,"""Film ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesátých let…","""film/pelisky/25517"""
"""ČT1""",2014-04-21 20:00:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR (1973). Klasická pohádka na motivy Boženy Němcové o krásné osiřelé dívce a kouzelném dárku. Scénář F. Pavlíček a V. Vorlíček. Kamera J. Ill…","""film/tri-orisky-pro-popelku/21407"""
"""ČT1""",2014-12-24 20:20:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesá…","""film/pelisky/25517"""
"""ČT1""",2015-08-02 20:00:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesá…","""film/pelisky/25517"""
"""ČT1""",2015-12-24 20:30:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR (1973). Opakování 27.12./ČT1. Klasická pohádka na motivy Boženy Němcové o krásné osiřelé dívce a kouzelném dárku. Scénář F. Pavlíček a V. V…","""film/tri-orisky-pro-popelku/21407"""
"""ČT1""",2015-12-24 21:55:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna z nejoblíbenějších filmových komedií s řadou hereckých hvězd o dětech a jejich rodičích, odehrávající se na konci šedesá…","""film/pelisky/25517"""


In [259]:
df.filter(pl.col("nazev") == "Pelíšky").group_by("kanal").len()

kanal,len
cat,u32
"""ČT1""",22


In [417]:
vysledky.columns

['index',
 'nazev',
 'Estimate',
 'Std. Error',
 'z value',
 'Pr(>|z|)',
 'low',
 'up',
 'csfd',
 'pocet_hodnoceni',
 'zkratka',
 'url',
 'format',
 'id_puvodni',
 'nazev_right',
 'zanry',
 'zeme',
 'rok',
 'delka',
 'rezie',
 'scenar',
 'hraji',
 'anotace',
 'hodnoceni',
 'pocet_hodnoceni_right',
 'id',
 'weighted_avg_kolikateho',
 'puvod']